In [1]:
# --- Imoort libraries ---
import os
import torchaudio
import torch
import subprocess
import tempfile
import pandas as pd
from transformers import AutoFeatureExtractor, WavLMModel, AutoTokenizer, RobertaModel

In [2]:
# --- Configuration ---
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "MELD.Raw", "train")

MP4_DIR = os.path.join(DATA_DIR, "train_splits")  # Audio files
CSV_FILE = os.path.join(DATA_DIR, "train_sent_emo.csv")  # Text utterances
COMBINEd_DIR =os.path.join(DATA_DIR, "train_combined")  # Combined Audio files

print(MP4_DIR)
print(CSV_FILE)
print(COMBINEd_DIR)


d:\works\6759\IFT-6759-Advanced-ML-Project\MELD.Raw\train\train_splits
d:\works\6759\IFT-6759-Advanced-ML-Project\MELD.Raw\train\train_sent_emo.csv
d:\works\6759\IFT-6759-Advanced-ML-Project\MELD.Raw\train\train_combined


In [3]:
# --- Models and Tokenizers ---
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/wavlm-base-plus")
wavlm_model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus")

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
roberta_model = RobertaModel.from_pretrained("roberta-base")

device = "mps" if torch.backends.mps.is_available() else "cpu"
wavlm_model.to(device)
roberta_model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [4]:
# Load CSV file
df = pd.read_csv(CSV_FILE)
df.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [5]:
df.tail()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
9984,10474,You or me?,Chandler,neutral,neutral,1038,13,2,3,"00:00:48,173","00:00:50,799"
9985,10475,"I got it. Uh, Joey, women don't have Adam's ap...",Ross,neutral,neutral,1038,14,2,3,"00:00:51,009","00:00:53,594"
9986,10476,"You guys are messing with me, right?",Joey,surprise,positive,1038,15,2,3,"00:01:00,518","00:01:03,520"
9987,10477,Yeah.,All,neutral,neutral,1038,16,2,3,"00:01:05,398","00:01:07,274"
9988,10478,"That was a good one. For a second there, I was...",Joey,joy,positive,1038,17,2,3,"00:01:08,401","00:01:12,071"


In [6]:
import os
from pydub import AudioSegment
from collections import defaultdict

# Get all .mp4 audio files
audio_files = [f for f in os.listdir(MP4_DIR) if f.endswith(".mp4")]

# Group files by dialogue number
dialogue_dict = defaultdict(list)
for file in audio_files:
    parts = file.split("_")  # Extract dialogue number
    if len(parts) >= 2:
        dialogue_num = parts[0]  # First part is the dialogue ID
        dialogue_dict[dialogue_num].append(file)

# Sort files by utterance ID within each dialogue
for dialogue in dialogue_dict:
    dialogue_dict[dialogue].sort(key=lambda x: int(x.split("_")[1].split(".")[0][3:] ))



In [7]:
# Output folder for combined audio
os.makedirs(COMBINEd_DIR, exist_ok=True)

In [ ]:
# Merge audio files for each dialogue
# dialog 125 has error
for dialogue_num, files in dialogue_dict.items(): 
    try:
        combined_audio = AudioSegment.empty()
        # Convert first audio file to mono/stereo (if needed)
        combined_audio = combined_audio.set_channels(1)   
        
        for file in files:
            file_path = os.path.join(MP4_DIR, file)  
            audio = AudioSegment.from_file(file_path, format="mp4")  # Load .mp4 file     
            
            # Convert to the same number of channels as combined_audio
            audio = audio.set_channels(combined_audio.channels)
            combined_audio += audio  # Concatenate audio files

        output_path = os.path.join(COMBINEd_DIR, f"{dialogue_num}_combined.mp4")
        combined_audio.export(output_path, format="mp4")  # Save as .mp4
        print(f"Saved: {output_path}")

    except Exception as e:
        print(f"Error processing dialogue {dialogue_num}: {e}")
        continue  # Continue with the next dialogue
print("All dialogues merged successfully!")

In [8]:
'''
V1
Save the pre-processed data (MELD)
Save by conversation (every utterances except last one + label of last one)
*V1: raw conversation without using previous labels
*Add index for each utterance embedding
'''
# Merge uttence into one row

# Extract the last emotion for each dialogue
last_emotion = df.groupby('Dialogue_ID').apply(lambda x: x.iloc[-1]['Emotion']).reset_index(name='Last_Emotion')

# Remove the last utterance of each dialogue
df_filtered = df.groupby('Dialogue_ID').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)


# Group by Dialogue ID and concatenate utterances
df_combined_v1 = df.groupby('Dialogue_ID').agg({
    'Utterance': lambda x: ' '.join(x),  # Combine utterances into one string
    'Emotion': lambda x: list(x),  # Keep emotions as a list
    #'Sentiment': lambda x: list(x)  # Keep sentiments as a list (optional)
}).reset_index()

# Merge the last emotion back into df_combined
df_combined_v1 = df_combined_v1.merge(last_emotion, on='Dialogue_ID', how='left')

# Add the 'audio_name' column based on Dialogue_ID
df_combined_v1['audio_name'] = df_combined_v1['Dialogue_ID'].apply(lambda x: f"dia{x}_combined.mp4")

# Save the processed dataset
df_combined_v1.to_csv("MELD_combined_v1.csv", index=False)
print("Processed dataset saved as MELD_combined_v1.csv")


Processed dataset saved as MELD_combined_v1.csv


C:\Users\liyua\AppData\Local\Temp\ipykernel_7808\2729715904.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_emotion = df.groupby('Dialogue_ID').apply(lambda x: x.iloc[-1]['Emotion']).reset_index(name='Last_Emotion')
C:\Users\liyua\AppData\Local\Temp\ipykernel_7808\2729715904.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df.groupby('Dialogue_ID').apply(lambda x: x.iloc[:-1]).re

In [9]:
df_combined_v1.head()

,Dialogue_ID,Utterance,Emotion,Last_Emotion,audio_name
0,0,also I was the point person on my companys tr...,"[neutral, neutral, neutral, neutral, surprise,...",neutral,dia0_combined.mp4
1,1,But then who? The waitress I went out with las...,"[surprise, sadness, surprise, fear, neutral, n...",neutral,dia1_combined.mp4
2,2,"Hey, Mon. Hey-hey-hey. You wanna hear somethin...","[neutral, neutral, joy, sadness, surprise, neu...",joy,dia2_combined.mp4
3,3,"Oh my God, oh my God! Poor Monica! What, what,...","[surprise, surprise, surprise, neutral, neutra...",neutral,dia3_combined.mp4
4,4,"Hey! Hi! What are you doing here? Ah y'know, t...","[surprise, joy, surprise, neutral, neutral, ne...",neutral,dia4_combined.mp4


In [10]:
df_combined_v1.tail()

,Dialogue_ID,Utterance,Emotion,Last_Emotion,audio_name
1033,1034,"Hey, Mon, I was just doing the dishes! Hey! Oh...","[neutral, joy, surprise]",surprise,dia1034_combined.mp4
1034,1035,What about me?! You-you just said I could! Im...,"[anger, neutral, anger, neutral, neutral, ange...",joy,dia1035_combined.mp4
1035,1036,Hey-hey! Stanley! Hey-hey! You're leading man ...,"[joy, sadness, surprise, sadness, surprise, ne...",sadness,dia1036_combined.mp4
1036,1037,"Rachel, do you have any muffins left? Yeah, I ...","[neutral, neutral, neutral]",neutral,dia1037_combined.mp4
1037,1038,Hey. Hey! So how was Joan? I broke up with her...,"[neutral, joy, neutral, neutral, surprise, dis...",joy,dia1038_combined.mp4


In [12]:
'''
V2
Save the pre-processed data (MELD)
Save by conversation (every utterances except last one + label of last one)
*V2: add label as text at the end of each conversation before feeding it to WaveLM/RoBERTa
'''
# Merge uttence into one row

# Extract the last emotion for each dialogue
last_emotion = df.groupby('Dialogue_ID').apply(lambda x: x.iloc[-1]['Emotion']).reset_index(name='Last_Emotion')

# Remove the last utterance of each dialogue
df_filtered = df.groupby('Dialogue_ID').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)


# Group by Dialogue ID and concatenate utterances
df_combined_v2 = df.groupby('Dialogue_ID').agg({
    'Utterance': lambda x: ' '.join(x),  # Combine utterances into one string
    'Emotion': lambda x: list(x),  # Keep emotions as a list
    #'Sentiment': lambda x: list(x)  # Keep sentiments as a list (optional)
}).reset_index()

# Merge the last emotion back into df_combined_v2
df_combined_v2 = df_combined_v2.merge(last_emotion, on='Dialogue_ID', how='left')

# Add the 'audio_name' column based on Dialogue_ID
df_combined_v2['audio_name'] = df_combined_v2['Dialogue_ID'].apply(lambda x: f"dia{x}_combined.mp4")

# Save the processed dataset
df_combined_v2.to_csv("MELD_combined_v2.csv", index=False)
print("Processed dataset saved as MELD_combined_v2.csv")


Processed dataset saved as MELD_combined_v2.csv


C:\Users\liyua\AppData\Local\Temp\ipykernel_7808\1463060540.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_emotion = df.groupby('Dialogue_ID').apply(lambda x: x.iloc[-1]['Emotion']).reset_index(name='Last_Emotion')
C:\Users\liyua\AppData\Local\Temp\ipykernel_7808\1463060540.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df.groupby('Dialogue_ID').apply(lambda x: x.iloc[:-1]).re

In [13]:
df_combined_v2.head()

,Dialogue_ID,Utterance,Emotion,Last_Emotion,audio_name
0,0,also I was the point person on my companys tr...,"[neutral, neutral, neutral, neutral, surprise,...",neutral,dia0_combined.mp4
1,1,But then who? The waitress I went out with las...,"[surprise, sadness, surprise, fear, neutral, n...",neutral,dia1_combined.mp4
2,2,"Hey, Mon. Hey-hey-hey. You wanna hear somethin...","[neutral, neutral, joy, sadness, surprise, neu...",joy,dia2_combined.mp4
3,3,"Oh my God, oh my God! Poor Monica! What, what,...","[surprise, surprise, surprise, neutral, neutra...",neutral,dia3_combined.mp4
4,4,"Hey! Hi! What are you doing here? Ah y'know, t...","[surprise, joy, surprise, neutral, neutral, ne...",neutral,dia4_combined.mp4


In [14]:
df_combined_v2.tail()

,Dialogue_ID,Utterance,Emotion,Last_Emotion,audio_name
1033,1034,"Hey, Mon, I was just doing the dishes! Hey! Oh...","[neutral, joy, surprise]",surprise,dia1034_combined.mp4
1034,1035,What about me?! You-you just said I could! Im...,"[anger, neutral, anger, neutral, neutral, ange...",joy,dia1035_combined.mp4
1035,1036,Hey-hey! Stanley! Hey-hey! You're leading man ...,"[joy, sadness, surprise, sadness, surprise, ne...",sadness,dia1036_combined.mp4
1036,1037,"Rachel, do you have any muffins left? Yeah, I ...","[neutral, neutral, neutral]",neutral,dia1037_combined.mp4
1037,1038,Hey. Hey! So how was Joan? I broke up with her...,"[neutral, joy, neutral, neutral, surprise, dis...",joy,dia1038_combined.mp4


In [19]:

import torch.nn.functional as F

def extract_audio_from_mp4(mp4_path):
    """Extracts audio from an MP4 file and converts it to WAV format."""
    temp_wav = tempfile.NamedTemporaryFile(suffix=".wav", delete=True).name
    command = ["ffmpeg", "-i", mp4_path, "-ac", "1", "-ar", "16000", "-y", temp_wav]
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return temp_wav


def extract_audio_features(mp4_path):
    """Extracts WavLM features while preserving sequence length."""
    wav_path = extract_audio_from_mp4(mp4_path)
    waveform, sample_rate = torchaudio.load(wav_path)

    if sample_rate != 16000:
        transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = transform(waveform)

    inputs = feature_extractor(waveform.squeeze(0), sampling_rate=16000, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = wavlm_model(**inputs)

    audio_features = outputs.last_hidden_state.cpu()  # (1, audio_seq_length, 768)

    # Resize sequence length to exactly 128
    audio_features = F.interpolate(audio_features.permute(0, 2, 1), size=128, mode="linear").permute(0, 2, 1)


    return audio_features  # Shape: (1, audio_seq_length resize to 128, 768) [1, 128, 768])



In [21]:
def extract_text_features(utterance):
    """Extracts RoBERTa features while preserving sequence length."""
    inputs = tokenizer(utterance, truncation=True, max_length=128, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = roberta_model(**inputs)
    
    text_features = outputs.last_hidden_state.cpu()

    # Resize sequence length to exactly 128
    text_features = F.interpolate(text_features.permute(0, 2, 1), size=128, mode="linear").permute(0, 2, 1)

    return text_features  # Shape: (1, text_seq_length resize to 128, 768)  [1, 128, 768])


In [22]:
import torch.nn.functional as F
def combine_audio_text_features(audio_features, text_features):
    # """Aligns sequence length and combines features."""
    # #seq_length = min(audio_features.shape[1], text_features.shape[1])
    # audio_features = audio_features[:, :seq_length, :]
    # text_features = text_features[:, :seq_length, :]


    # Pads shorter sequence and combines features."""
    # max_seq_length = max(audio_features.shape[1], text_features.shape[1])
    
    # # Pad audio features
    # if audio_features.shape[1] < max_seq_length:
    #     pad_size = max_seq_length - audio_features.shape[1]
    #     audio_features = F.pad(audio_features, (0, 0, 0, pad_size))  # Pad in sequence dimension

    # # Pad text features
    # if text_features.shape[1] < max_seq_length:
    #     pad_size = max_seq_length - text_features.shape[1]
    #     text_features = F.pad(text_features, (0, 0, 0, pad_size))  # Pad in sequence dimension

    # print("audio_features.shape",audio_features.shape)
    # print("text_features.shape",text_features.shape)
    # print("after combined ",torch.cat((audio_features, text_features), dim=-1).shape)

    return torch.cat((audio_features, text_features), dim=-1)  # Shape: (1, 128, 1536)  


In [26]:
# v1 version
import pickle
import numpy as np
import torch.nn.functional as F

# Dictionary to store combined features
combined_features_list_v1 = []

# Process each dialogue
for index, row in df_combined_v1.iterrows():     
    try:
        utterance = row["Utterance"]
        mp4_filename = row["audio_name"]
        mp4_path = os.path.join(COMBINEd_DIR, mp4_filename)

        if not os.path.exists(mp4_path):
            print(f"Skipping missing file: {mp4_path}")
            continue

        print(f"Processing {mp4_filename}...")

        audio_features = extract_audio_features(mp4_path)
        text_features = extract_text_features(utterance)

        combined_features = combine_audio_text_features(audio_features, text_features)         
        combined_features_list_v1.append(combined_features)
        # Ensure list is not empty
        if not combined_features_list_v1:
            raise ValueError("Error: combined_features_list_v1 is empty!")
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue  # Continue with the next dialogue


# Ensure list is not empty
if not combined_features_list_v1:
    raise ValueError("Error: combined_features_list_v1 is empty!")

# Check if all items are tensors
for i, item in enumerate(combined_features_list_v1):
    if not isinstance(item, torch.Tensor):
        raise TypeError(f"Error: Item {i} is not a tensor! Found {type(item)}")


# # Find the maximum sequence length in the list
# max_seq_length = max(item.shape[1] for item in combined_features_list_v1)
# print(max_seq_length)
# # Pad all tensors to max_seq_length
# padded_features_list = [
#     F.pad(item, (0, 0, 0, max_seq_length - item.shape[1]))  # Pad sequence dimension
#     for item in combined_features_list_v1
# ]

# Convert list to a single tensor
final_tensor_conversion_v1 = torch.cat(combined_features_list_v1, dim=0)


print(f"Final tensor shape: {final_tensor_conversion_v1.shape}")  # Expected: (num_dialogues, 128, 1536)

# Define the file path
file_path_v1 = "final_tensor_conversition_v1.pkl"

# Save the tensor to a .pkl file
with open(file_path_v1, "wb") as f:
    pickle.dump(final_tensor_conversion_v1, f)

print(f"Tensor saved to {file_path_v1}")

Processing dia0_combined.mp4...
Processing dia1_combined.mp4...
Processing dia2_combined.mp4...
Processing dia3_combined.mp4...
Processing dia4_combined.mp4...
Processing dia5_combined.mp4...
Processing dia6_combined.mp4...
Processing dia7_combined.mp4...
Processing dia8_combined.mp4...
Processing dia9_combined.mp4...
Processing dia10_combined.mp4...
Processing dia11_combined.mp4...
Processing dia12_combined.mp4...
Processing dia13_combined.mp4...
Processing dia14_combined.mp4...
Processing dia15_combined.mp4...
Processing dia16_combined.mp4...
Processing dia17_combined.mp4...
Processing dia18_combined.mp4...
Processing dia19_combined.mp4...
Processing dia20_combined.mp4...
Processing dia21_combined.mp4...
Processing dia22_combined.mp4...
Processing dia23_combined.mp4...
Processing dia24_combined.mp4...
Processing dia25_combined.mp4...
Processing dia26_combined.mp4...
Processing dia27_combined.mp4...
Processing dia28_combined.mp4...
Processing dia29_combined.mp4...
Processing dia30_com

In [28]:
# v2 version
import pickle
import numpy as np
import torch.nn.functional as F

# Dictionary to store combined features
combined_features_list_v2 = []

# Process each dialogue
# Process each utterance
for index, row in df_combined_v2.iterrows(): 
    try:
        utterance = row["Utterance"]
        mp4_filename = row["audio_name"]
        mp4_path = os.path.join(COMBINEd_DIR, mp4_filename)

        if not os.path.exists(mp4_path):
            print(f"Skipping missing file: {mp4_path}")
            continue

        print(f"Processing {mp4_filename}...")

        audio_features = extract_audio_features(mp4_path)
        text_features = extract_text_features(utterance)

        combined_features = combine_audio_text_features(audio_features, text_features)         
        combined_features_list_v2.append(combined_features)
        
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue  # Continue with the next dialogue



# # Find the maximum sequence length in the list
# max_seq_length = max(item.shape[1] for item in combined_features_list_v2)

# # Pad all tensors to max_seq_length
# padded_features_list = [
#     F.pad(item, (0, 0, 0, max_seq_length - item.shape[1]))  # Pad sequence dimension
#     for item in combined_features_list_v2
# ]

# Convert list to a single tensor
final_tensor_conversion_v2 = torch.cat(combined_features_list_v2, dim=0)

print(f"Final tensor shape: {final_tensor_conversion_v1.shape}")  # Expected: (num_dialogues, max_seq_length, 1536)


# Define the file path
file_path_v2 = "final_tensor_conversition_v2.pkl"

# Save the tensor to a .pkl file
with open(file_path_v2, "wb") as f:
    pickle.dump(final_tensor_conversion_v2, f)

print(f"Tensor saved to {file_path_v2}")



Processing dia0_combined.mp4...


d:\miniconda3\Lib\site-packages\torch\nn\functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Processing dia1_combined.mp4...
Processing dia2_combined.mp4...
Processing dia3_combined.mp4...
Processing dia4_combined.mp4...
Processing dia5_combined.mp4...
Processing dia6_combined.mp4...
Processing dia7_combined.mp4...
Processing dia8_combined.mp4...
Processing dia9_combined.mp4...
Processing dia10_combined.mp4...
Processing dia11_combined.mp4...
Processing dia12_combined.mp4...
Processing dia13_combined.mp4...
Processing dia14_combined.mp4...
Processing dia15_combined.mp4...
Processing dia16_combined.mp4...
Processing dia17_combined.mp4...
Processing dia18_combined.mp4...
Processing dia19_combined.mp4...
Processing dia20_combined.mp4...
Processing dia21_combined.mp4...
Processing dia22_combined.mp4...
Processing dia23_combined.mp4...
Processing dia24_combined.mp4...
Processing dia25_combined.mp4...
Processing dia26_combined.mp4...
Processing dia27_combined.mp4...
Processing dia28_combined.mp4...
Processing dia29_combined.mp4...
Processing dia30_combined.mp4...
Processing dia31_co